# Function optimization

In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
import time
from IPython.display import clear_output

In [50]:
df = pd.read_csv("../data/BTCUSDT-volume-bars.csv", index_col=0, parse_dates=True)
ind = pd.read_csv("../data/vol.csv", index_col=0, parse_dates=True, squeeze=True)

In [54]:
df = df[-1000:]

In [44]:
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.00015306472778320312 seconds ---


In [88]:
def get_t_events(g_raw, h, m):
    """
    Implementation of the symmetric CUSUM filter seen in chapter 2.5.2
    
    Arguments:
    g_raw -- the raw time series to filter (possibly pandas series)
    h -- threshold pandas series vol
    m -- minimum threshold (float, ex: 0.02)
    
    Purpose:
    The CUSUM filter is a quality-control method, designed to detect a shift in the mean value of a measured quantity away from a target value. 
    """
    h = h.map(lambda x: x if x>=m else m)
    t_events, s_pos, s_neg = [], 0, 0
    diff = np.log(g_raw).diff().dropna()
    acc = 0
    for i in diff.index[1:]:
        print(i)
        start_time = time.time()
        try:
            pos, neg = float(s_pos+diff.loc[i]), float(s_neg+diff.loc[i])
        except Exception as e:
            print(e)
            print(s_pos+diff.loc[i], type(s_pos+diff.loc[i]))
            print(s_neg+diff.loc[i], type(s_neg+diff.loc[i]))
            break
            
        print("--- %s seconds 1---" % (time.time() - start_time))
        start_time = time.time()
        
        s_pos, s_neg=max(0., pos), min(0., neg)
        
        print("--- %s seconds 2---" % (time.time() - start_time))
        start_time = time.time()
        
#         th = h[h.index.get_loc(i, method="pad")]
        
        try: 
            th = h.loc[i]
        except:
            th = h[h.index.get_loc(i, method="pad")]
            pass
        
        print("--- %s seconds 3---" % (time.time() - start_time))
        start_time = time.time()
        
        if s_neg<-th:
            s_neg=0;t_events.append(i)
        elif s_pos>th:
            s_pos=0;t_events.append(i)
        print("--- %s seconds 4---" % (time.time() - start_time))
        acc += 1
    return pd.DatetimeIndex(t_events), acc

In [45]:
def get_t_events(g_raw, h, m):
    """
    Implementation of the symmetric CUSUM filter seen in chapter 2.5.2
    
    Arguments:
    g_raw -- the raw time series to filter (possibly pandas series)
    h -- threshold pandas series vol
    m -- minimum threshold (float, ex: 0.02)
    
    Purpose:
    The CUSUM filter is a quality-control method, designed to detect a shift in the mean value of a measured quantity away from a target value. 
    """
    h = h.map(lambda x: x if x>=m else m)
    t_events, s_pos, s_neg = [], 0, 0
    diff = np.log(g_raw).diff().dropna()
    for i in diff.index[1:]:
        try:
            pos, neg = float(s_pos+diff.loc[i]), float(s_neg+diff.loc[i])
        except Exception as e:
            print(e)
            print(s_pos+diff.loc[i], type(s_pos+diff.loc[i]))
            print(s_neg+diff.loc[i], type(s_neg+diff.loc[i]))
            break

        
        s_pos, s_neg=max(0., pos), min(0., neg)
        
        
        try: 
            th = h.loc[i]
        except:
            th = h[h.index.get_loc(i, method="pad")]

        if s_neg<-th:
            s_neg=0;t_events.append(i)
        elif s_pos>th:
            s_pos=0;t_events.append(i)
        
    return pd.DatetimeIndex(t_events)

In [46]:
df.index[2]

Timestamp('2021-06-13 03:10:14.601000')

In [89]:
start_time = time.time()
t_events, acc = get_t_events(df.close,h=ind, m=0.02)
print("--- %s seconds 2---" % (time.time() - start_time))
t_events, acc

2021-06-19 14:34:45.431000
--- 0.00043320655822753906 seconds 1---
--- 2.86102294921875e-06 seconds 2---
--- 7.987022399902344e-05 seconds 3---
--- 3.0994415283203125e-06 seconds 4---
2021-06-19 14:34:45.687000
--- 9.083747863769531e-05 seconds 1---
--- 2.1457672119140625e-06 seconds 2---
--- 5.984306335449219e-05 seconds 3---
--- 9.5367431640625e-07 seconds 4---
2021-06-19 14:34:47.647000
--- 0.00013303756713867188 seconds 1---
--- 2.1457672119140625e-06 seconds 2---
--- 6.794929504394531e-05 seconds 3---
--- 1.9073486328125e-06 seconds 4---
--- 3.8022170066833496 seconds 2---


(DatetimeIndex([], dtype='datetime64[ns]', freq=None), 3)

In [87]:
th = ind.loc["2021-06-19 14:34:47.647000"]
print(th)

0.0127811714789338


In [76]:
df

,open,high,low,close,volume
time,,,,,
2021-06-19 14:34:30.704,36192.96,36200.00,36192.96,36200.00,5.256469
2021-06-19 14:34:38.427,36199.87,36200.00,36196.38,36200.00,14.791975
2021-06-19 14:34:45.431,36200.00,36200.80,36200.00,36200.00,5.005186
2021-06-19 14:34:45.687,36200.00,36208.00,36200.00,36208.00,5.069611
2021-06-19 14:34:47.647,36208.00,36209.89,36200.00,36200.00,5.450086
...,...,...,...,...,...
2021-06-19 16:27:04.502,35831.71,35844.86,35831.71,35841.54,5.539350
2021-06-19 16:27:20.667,35841.57,35841.58,35822.03,35823.32,5.033671
2021-06-19 16:27:26.627,35823.32,35833.69,35823.32,35826.83,5.425236


In [71]:
th

0.0127811714789338